In [1]:
import pandas as pd
import numpy as np
from numpy import NaN
from itertools import chain
import json

In [2]:
# напрямую с файлами
ac = pd.read_csv('results/raw/1С-аналитик.csv', sep=';', encoding='utf-8')
ds = pd.read_csv('results/raw/Data Scientist.csv', sep=';', encoding='utf-8')
do = pd.read_csv('results/raw/DataOps-инженер.csv', sep=';', encoding='utf-8')
ab = pd.read_csv('results/raw/Аналитик BI.csv', sep=';', encoding='utf-8')
ad = pd.read_csv('results/raw/Аналитик данных.csv', sep=';', encoding='utf-8')
dj = pd.read_csv('results/raw/Дата-журналист.csv', sep=';', encoding='utf-8')
de = pd.read_csv('results/raw/Дата-инженер.csv', sep=';', encoding='utf-8')
ma = pd.read_csv('results/raw/Маркетинговый аналитик.csv', sep=';', encoding='utf-8')
pa = pd.read_csv('results/raw/Продуктовый аналитик.csv', sep=';', encoding='utf-8')
sa = pd.read_csv('results/raw/Системный аналитик.csv', sep=';', encoding='utf-8')
fa = pd.read_csv('results/raw/Финансовый аналитик.csv', sep=';', encoding='utf-8')
ba = pd.read_csv('results/raw/Бизнес-аналитик.csv', sep=';', encoding='utf-8')


In [3]:
# добавление меток широта-долгота
with open('geo_rus.json', encoding='utf-8') as geo:
    cities = json.load(geo)

def lat(row):
    for el in cities:
        if row == el['city']:
            return float(el['lat'])
        
def lon(row):
    for el in cities:
        if row == el['city']:
            return float(el['lon'])


In [4]:
full = pd.concat([ds, do, ab, ad, dj, de, ma, sa, ac, fa, ba, pa])

full['lat'] = full['city'].apply(lat)
full['lon'] = full['city'].apply(lon)
full = full[['id', 'spec', 'job_title', 'city', 'lat', 'lon', 
         'employer', 'schedule', 'skills', 'experience', 
         'published', 'salary_from', 'salary_to', 'currency']]

full.to_csv('vacancies.csv', index=False)

In [5]:
ac_sk = ['1c|1с', 'agile|scrum|kanban', 'управление', 'erp', 'отчет',
         'анализ', 'тз', 'srs', 'моделирование', 'bpmn', 'тестирование',
         'документ']

ds_sk = ['анализ', 'sheets', 'статистика', 'data studio', 'python', 'sql',
         'ml', 'machine learning', 'сбор', 'визуализация', 'исследование',
         'нейронные сети', 'нейросети', 'временные ряды', 'алгоритм',
         'sklearn', 'tensorflow', 'pytorch','cv', 'gan', 'nlp']

de_do_sk = ['sql', 'базы данных', 'etl', 'power bi|powerbi', 'python', 
            'pandas', 'numpy', 'hadoop', 'mapreduce', 'apache', 'kafka', 
            'spark', 'airflow', 'clickhouse', 'yandex cloud', 'docker', 
            'kubernetes', 'ansible', 'gitlab ci', 'mlflow']

ab_sk = ['анализ', 'sheets', 'excel', 'статистика', 'визуализация', 
         'data studio', 'power bi|powerbi', 'pivot', 'datalens', 'tableau',
         'сбор', 'sql', 'ssas']

ad_sk = ['анализ', 'sheets', 'excel', 'статистика', 'визуализация',
         'data studio', 'python', 'jupyter notebook', 'numpy', 
         'pandas', 'сбор', 'sql', 'ml', 'machine learning', 'hadoop']

dj_sk = ['анализ', 'сбор', 'очистка', 'визуализация', 'построение карт', 'сторителлинг',
         'python', 'pandas', 'статистика', 'tableau']

ma_sk = ['анализ', 'sheets', 'статистика', 'визуализация', 'google looker studio',
         'метрики', 'юнит-экономика', 'поиск точек роста', 'RFM-анализ', 
         'когортный анализ', 'веб-аналитика', 'яндекс метрика', 'google analitics',
         'ab', 'sql', 'power bi|powerbi', 'python', 'numpy', 'pandas', 'api']

pa_sk = ['анализ', 'юнит-экономика', 'ab', 'sql', 'tableau', 'power bi|powerbi', 
         'python', 'numpy', 'pandas', 'исследов']

sa_sk = ['разработка по', 'agile', 'scrum', 'моделирование бизнес-процессов',
         'нотации', 'проектирование', 'сбор', 'описание', 'визуализация',
         'прототипирование', 'figma', 'sql', 'rest api', 'soap', 'open api',
         'проектная документация', 'cjm', 'тз', 'спецификация', 'SRS', 'bcm']

fa_sk = ['1с', 'бухгалтер', 'финансовая', 'анализ', 'управление', 'статистика', 
         'план-фактный анализ', 'планирование', 'моделирование', 'презентац',
         'powerpoint', 'excel', 'визуализация']

ba_sk = ['моделирование', 'анализ', 'оптимизация', 'отчетност', 'сбор',
         'tableau', 'bi', 'исследовани', 'agile|scrum|kanban', 'тз', 'документ']


### Топ навыков

#### 1 / 12 Аналитик данных, подготовка к конкату

In [6]:
ad.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ad.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ad.skills.str.split(',').map(len)

ad = pd.DataFrame({'id': np.repeat(ad['id'], lens),
                    'spec': np.repeat(ad['spec'], lens),
                    'experience': np.repeat(ad['experience'], lens),
                    'skills': chainer(ad['skills'])})

In [7]:
ad = ad.loc[ad.skills.str.contains('|'.join(ad_sk))]
ad.skills = ad.skills.str.strip()

ad.loc[(ad.skills.str.contains('sql'), 'skills')] = 'sql'
ad.loc[(ad.skills.str.contains('excel'), 'skills')] = 'excel'
ad.loc[(ad.skills.str.contains('machine learning'), 'skills')] = 'ml'
ad.loc[(ad.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'

#### 2 / 12 Data Scientist, подготовка к конкату

In [8]:
ds.drop(['job_title', 'city', 'employer', 'schedule', 
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ds.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ds.skills.str.split(',').map(len)

ds = pd.DataFrame({'id': np.repeat(ds['id'], lens),
                    'spec': np.repeat(ds['spec'], lens),
                    'experience': np.repeat(ds['experience'], lens),
                    'skills': chainer(ds['skills'])})

In [9]:
ds = ds.loc[ds.skills.str.contains('|'.join(ds_sk))]
ds.skills = ds.skills.str.strip()

ds.loc[(ds.skills.str.contains('sql'), 'skills')] = 'sql'
ds.loc[(ds.skills.str.contains('excel'), 'skills')] = 'excel'
ds.loc[(ds.skills.str.contains('machine learning'), 'skills')] = 'ml'
ds.loc[(ds.skills.str.contains('ml'), 'skills')] = 'ml'
ds.loc[(ds.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'

#### 3 / 12 Аналитик BI, подготовка к конкату

In [10]:
ab.drop(['job_title', 'city', 'employer', 'schedule', 
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ab.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ab.skills.str.split(',').map(len)

ab = pd.DataFrame({'id': np.repeat(ab['id'], lens),
                    'spec': np.repeat(ab['spec'], lens),
                    'experience': np.repeat(ab['experience'], lens),
                    'skills': chainer(ab['skills'])})

In [11]:
ab = ab.loc[ab.skills.str.contains('|'.join(ab_sk))]
ab.skills = ab.skills.str.strip()

ab.loc[(ab.skills.str.contains('sql'), 'skills')] = 'sql'
ab.loc[(ab.skills.str.contains('excel'), 'skills')] = 'excel'
ab.loc[(ab.skills.str.contains('bi'), 'skills')] = 'power bi'
ab.loc[(ab.skills.str.contains('pivot'), 'skills')] = 'pivot'
ab.loc[(ab.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'

#### 4 / 12 Системный аналитик, подготовка к конкату

In [12]:
sa.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
sa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = sa.skills.str.split(',').map(len)

sa = pd.DataFrame({'id': np.repeat(sa['id'], lens),
                    'spec': np.repeat(sa['spec'], lens),
                    'experience': np.repeat(sa['experience'], lens),
                    'skills': chainer(sa['skills'])})

In [13]:
sa = sa.loc[sa.skills.str.contains('|'.join(sa_sk))]
sa.skills = sa.skills.str.strip()

sa.loc[(sa.skills.str.contains('sql'), 'skills')] = 'sql'
sa.loc[(sa.skills.str.contains('agile'), 'skills')] = 'agile'
sa.loc[(sa.skills.str.contains('тз'), 'skills')] = 'написание тз'
sa.loc[(sa.skills.str.contains('soap'), 'skills')] = 'soap'
sa.loc[(sa.skills.str.contains('моделирование | проектирование'), 'skills')] = 'проектирование (интерфейсов, api, т.д.)'
sa.loc[(sa.skills.str.contains('сбор требований'), 'skills')] = 'сбор требований'

#### 5 / 12 Маркетинговый аналитик, подготовка к конкату

In [14]:
ma.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ma.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ma.skills.str.split(',').map(len)

ma = pd.DataFrame({'id': np.repeat(ma['id'], lens),
                    'spec': np.repeat(ma['spec'], lens),
                    'experience': np.repeat(ma['experience'], lens),
                    'skills': chainer(ma['skills'])})

In [15]:
ma = ma.loc[ma.skills.str.contains('|'.join(ma_sk))]
ma.skills = ma.skills.str.strip()

ma.loc[(ma.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'
ma.loc[(ma.skills.str.contains('sql'), 'skills')] = 'sql'
ma.loc[(ma.skills.str.contains('статистика'), 'skills')] = 'статистика'

#### 6 / 12 Дата-журналист, подготовка к конкату

In [16]:
dj.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
dj.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = dj.skills.str.split(',').map(len)

dj = pd.DataFrame({'id': np.repeat(dj['id'], lens),
                    'spec': np.repeat(dj['spec'], lens),
                    'experience': np.repeat(dj['experience'], lens),
                    'skills': chainer(dj['skills'])})

In [ ]:
dj = dj.loc[dj.skills.str.contains('|'.join(dj_sk))]
dj.skills = dj.skills.str.strip()

dj.loc[(dj.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'
dj.loc[(dj.skills.str.contains('сбор'), 'skills')] = 'сбор данных'
dj.loc[(dj.skills.str.contains('визуализация'), 'skills')] = 'визуализация данных'
dj.loc[(dj.skills.str.contains('storytelling'), 'skills')] = 'сторителлинг'
dj.loc[(dj.skills.str.contains('story telling'), 'skills')] = 'сторителлинг'

#### 7-8 / 12 Дата-инженер | DataOps-инженер, подготовка к конкату

In [18]:
#DataOps
do.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
do.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = do.skills.str.split(',').map(len)

do = pd.DataFrame({'id': np.repeat(do['id'], lens),
                    'spec': np.repeat(do['spec'], lens),
                    'experience': np.repeat(do['experience'], lens),
                    'skills': chainer(do['skills'])})

In [19]:
do = do.loc[do.skills.str.contains('|'.join(de_do_sk))]
do.skills = do.skills.str.strip()

do.skills.unique()

# do.skills.loc[(do.skills.str.contains('...'))] = '...'
# do.skills.loc[(do.skills.str.contains('...'))] = '...'

array(['python', 'docker', 'clickhouse', 'airflow', 'kubernetes', 'kafka',
       'nosql', 'hadoop'], dtype=object)

In [20]:
# дата-инженер
de.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
de.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = de.skills.str.split(',').map(len)

de = pd.DataFrame({'id': np.repeat(de['id'], lens),
                    'spec': np.repeat(de['spec'], lens),
                    'experience': np.repeat(de['experience'], lens),
                    'skills': chainer(de['skills'])})

In [21]:
de = de.loc[de.skills.str.contains('|'.join(de_do_sk))]
de.skills = de.skills.str.strip()

de.loc[(de.skills.str.contains('sql'), 'skills')] = 'sql'
de.loc[(de.skills.str.contains('bi'), 'skills')] = 'power bi'
de.loc[(de.skills.str.contains('airflow'), 'skills')] = 'airflow'
de.loc[(de.skills.str.contains('nifi|ni-fi'), 'skills')] = 'nifi'

#### 9 / 12 Финансовый аналитик, подготовка к конкату

In [22]:
fa.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
fa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = fa.skills.str.split(',').map(len)

fa = pd.DataFrame({'id': np.repeat(fa['id'], lens),
                    'spec': np.repeat(fa['spec'], lens),
                    'experience': np.repeat(fa['experience'], lens),
                    'skills': chainer(fa['skills'])})

In [23]:
fa = fa.loc[fa.skills.str.contains('|'.join(fa_sk))]
fa.skills = fa.skills.str.strip()

fa.loc[(fa.skills.str.contains('1с|1c'), 'skills')] = '1с (предприятие, бухгалтерия)'
fa.loc[(fa.skills.str.contains('excel'), 'skills')] = 'excel'
fa.loc[(fa.skills.str.contains('powerpoint|презент'), 'skills')] = 'powerpoint'
fa.loc[(fa.skills.str.contains('анали'), 'skills')] = 'анализ (данных, финансовый, т.д.)'
fa.loc[(fa.skills.str.contains('управление'), 'skills')] = 'управление (рисками, затратами, т.д.)'
fa.loc[(fa.skills.str.contains('планирование'), 'skills')] = 'планирование (финансовое, инвестиционное, т.д.)'
fa.loc[(fa.skills.str.contains('моделирование'), 'skills')] = 'моделирование (финансовое, экономическое, т.д.)'
fa.loc[(fa.skills.str.contains('бухгал'), 'skills')] = 'бухучёт'
fa.loc[(fa.skills.str.contains('статистика'), 'skills')] = 'статистика'
fa.loc[(fa.skills.str.contains('визуализация'), 'skills')] = 'визуализация'

#### 10 / 12 1С-аналитик, подготовка к конкату

In [24]:
ac.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ac.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ac.skills.str.split(',').map(len)

ac = pd.DataFrame({'id': np.repeat(ac['id'], lens),
                    'spec': np.repeat(ac['spec'], lens),
                    'experience': np.repeat(ac['experience'], lens),
                    'skills': chainer(ac['skills'])})

In [25]:
ac = ac.loc[ac.skills.str.contains('|'.join(ac_sk))]
ac.skills = ac.skills.str.strip()

ac.loc[(ac.skills.str.contains('1с|1c'), 'skills')] = '1с (предприятие, бухгалтерия)'
ac.loc[(ac.skills.str.contains('отчетность'), 'skills')] = 'отчетность (налоговая, бухгалтерская)'
ac.loc[(ac.skills.str.contains('powerpoint'), 'skills')] = 'powerpoint'
ac.loc[(ac.skills.str.contains('документ'), 'skills')] = 'проектная документация'

#### 11 / 12 Продуктовый аналитик, подготовка к конкату

In [26]:
pa.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
pa.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = pa.skills.str.split(',').map(len)

pa = pd.DataFrame({'id': np.repeat(pa['id'], lens),
                    'spec': np.repeat(pa['spec'], lens),
                    'experience': np.repeat(pa['experience'], lens),
                    'skills': chainer(pa['skills'])})

In [28]:
pa = pa.loc[pa.skills.str.contains('|'.join(pa_sk))]
pa.skills = pa.skills.str.strip()

pa.loc[(pa.skills.str.contains('sql'), 'skills')] = 'sql'
pa.loc[(pa.skills.str.contains('анализ|analys'), 'skills')] = 'анализ (данных, финансовый, т.д.)'
pa.loc[(pa.skills.str.contains('bi'), 'skills')] = 'power bi'
pa.loc[(pa.skills.str.contains('исследов'), 'skills')] = 'проведение исследований'

#### 12 / 12 Бизнес-аналитик, подготовка к конкату

In [29]:
ba.drop(['job_title', 'city', 'employer', 'schedule',
         'published', 'salary_from', 'salary_to', 'currency'], 
         axis=1, inplace=True)
ba.skills.replace(NaN, '-', inplace=True)

def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = ba.skills.str.split(',').map(len)

ba = pd.DataFrame({'id': np.repeat(ba['id'], lens),
                    'spec': np.repeat(ba['spec'], lens),
                    'experience': np.repeat(ba['experience'], lens),
                    'skills': chainer(ba['skills'])})

In [30]:
ba = ba.loc[ba.skills.str.contains('|'.join(ba_sk))]
ba.skills = ba.skills.str.strip()

ba.loc[(ba.skills.str.contains('анализ'), 'skills')] = 'анализ (данных, финансовый, т.д.)'
ba.loc[(ba.skills.str.contains('моделирование'), 'skills')] = 'моделирование (финансовое, экономическое, т.д.)'
ba.loc[(ba.skills.str.contains('отчет'), 'skills')] = 'отчетность (налоговая, бухгалтерская)'
ba.loc[(ba.skills.str.contains('bi'), 'skills')] = 'bi-системы'
ba.loc[(ba.skills.str.contains('документ'), 'skills')] = 'проектная документация'
ba.loc[(ba.skills.str.contains('оптимизация'), 'skills')] = 'оптимизация процессов'
ba.loc[(ba.skills.str.contains('исследов'), 'skills')] = 'проведение исследований'
ba.loc[(ba.skills.str.contains('тз'), 'skills')] = 'составление тз'
ba.loc[(ba.skills.str.contains('сбор'), 'skills')] = 'сбор требований'
ba.loc[(ba.skills.str.contains('agile'), 'skills')] = 'agile'

In [ ]:
df = pd.concat([ds, do, ab, ad, dj, de, ma, sa, ac, fa, ba, pa])
df.skills.replace(NaN, '-', inplace=True)
df.to_excel('key_skills.xlsx', index=False)

### Сводная статистика

In [99]:
count = pd.read_excel('results/agg_stat.xlsx', sheet_name='count_of_vac')
dist = pd.read_excel('results/agg_stat.xlsx', sheet_name='exp_distribution')
m_sal = pd.read_excel('results/agg_stat.xlsx', sheet_name='median_sal')

In [118]:
from pathlib import Path
from datetime import datetime

p = Path(r'C:\Users\ePROkhorov\Documents\hh_API_new\results\raw').glob('**/*')
files = [x for x in p]

# добавить кол-во новых вакансий замесяц
for file in files:
    months = ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    month = ''
    year = datetime.now().year
    name = file.stem
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    for i, el in enumerate(months):
        if datetime.now().month == i + 2:
            month = el
    row = {'spec': name, 'year': year, 'month': month, 'count': len(df)}
    count = pd.concat([count, pd.DataFrame([row])])

# апдейтнуть распределение вакансий за год
for file in files:
    year = datetime.now().year
    name = file.stem  
    df = pd.read_csv(file, sep=';', encoding='utf-8')
    exp_new = (df.experience.value_counts()
               .to_frame()
               .reset_index()
               .rename({'experience': 'exp', 'count': 'distrib'}, axis=1))
    exp_old = dist.loc[(dist.spec == name) & (dist.year == year)][['exp', 'distrib']].reset_index(drop=True)
    merged_df = pd.merge(exp_old, exp_new, on=["exp"], how='outer', suffixes=("_test", "_new")).replace(np.nan, 0)
    merged_df["distrib"] = merged_df["distrib_test"] + merged_df["distrib_new"]
    merged_df.drop(['distrib_test', 'distrib_new'], axis=1, inplace=True)
    res = merged_df.distrib.astype(int).to_list()
    dist.loc[(dist.spec == name) & (dist.year == year), 'distrib'] = res
            
# апдейтнуть медианные зп за год
for file in files:
    year = datetime.now().year
    name = file.stem  
    df = pd.read_csv(file, sep=';', encoding='utf-8')  
    sal_new = np.array([df.salary_from.median(), df.salary_to.median()]).mean()
    ind = m_sal.loc[(m_sal.spec == name) & (m_sal.year == datetime.now().year)].index
    sal_old = m_sal.iloc[ind]['sal'].reset_index().sal.values[0]
    m_sal.loc[(m_sal.spec == name) & (m_sal.year == datetime.now().year), 'sal'] = np.array([sal_old, sal_new]).mean()
